# CNN Baseline Code

## 라이브러리 호출

In [5]:
import os
import tensorflow as tf
import keras

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling


from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, MaxPooling2D
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization

from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax


from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [6]:
tf.random.set_seed(42)

## 데이터 불러오기

In [7]:
# 150개 메뉴이름 리스트에 담기
small_list = []

for i in os.listdir('/Users/hanhyeongu/Desktop/codestates/CP1/Korea_Food_Image/train'):
    small_list.append(i)      

len(small_list)

150

In [8]:
train = image_dataset_from_directory(
    '/Users/hanhyeongu/Desktop/codestates/CP1/Korea_Food_Image/train',
    labels="inferred",
    label_mode="int",
    class_names=small_list,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=1,
    interpolation="bilinear"
)

val = image_dataset_from_directory(
    '/Users/hanhyeongu/Desktop/codestates/CP1/Korea_Food_Image/validation',
    labels="inferred",
    label_mode="int",
    class_names=small_list,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=1,
    interpolation="bilinear"
)

Found 90000 files belonging to 150 classes.
Found 30000 files belonging to 150 classes.


# Model 

In [9]:
# 사전학습모델(ResNet50)
def resnet(num_classes=150):
    resnet = ResNet50(weights='imagenet', include_top=False)
    for layer in resnet.layers:
        layer.trainable = False
    model = Sequential()
    model.add(resnet)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['acc']
    )

    return model


# 사전학습모델(VGG16)
def mobilenet(num_classes=150):
    mobile = MobileNetV2(weights='imagenet', include_top=False)
    for layer in mobile.layers:
        layer.trainable = False
    model = Sequential()
    model.add(mobile)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['acc']
    )

    return model


# custom model1
def cnn1(num_classes=150):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten()) 
    model.add(Dense(256,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(512,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['acc']
    )

    return model


# 미세조정
def fine_tunning(model):
    model.layers[1].trainable = True

    model.compile(optimizer=tf.keras.optimizers.SGD(0.001), #lower learning rate
                    loss='categorical_crossentropy',
                    metrics = ['accuracy'])

    return model


## Fit Model

In [10]:
model1_2 = resnet()

In [12]:
model1_2.fit(
    train,
    validation_data=val,
    batch_size=256,
    epochs=5
)

Epoch 1/5
 365/2813 [==>...........................] - ETA: 7:11 - loss: 1.6608 - acc: 0.5509

Invalid SOS parameters for sequential JPEG


 638/2813 [=====>........................] - ETA: 6:23 - loss: 1.6547 - acc: 0.5517

Invalid SOS parameters for sequential JPEG


 698/2813 [======>.......................] - ETA: 6:12 - loss: 1.6556 - acc: 0.5511

...

1765/2813 [=================>............] - ETA: 3:05 - loss: 1.2874 - acc: 0.6410

Invalid SOS parameters for sequential JPEG


2813/2813 [==============================] - ETA: 0s - loss: 1.2859 - acc: 0.6409

Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


2813/2813 [==============================] - 660s 234ms/step - loss: 1.2859 - acc: 0.6409 - val_loss: 1.8009 - val_acc: 0.5442


# Model 저장

In [13]:
model1_2.save('/Users/hanhyeongu/Desktop/codestates/CP1/Yam-Pick_DS/model/resnet50_ver2.h5')